# Loading Modules

In [38]:
import requests, json
from pprint import pprint 
import language_check
import spacy
import nltk
from spacy import displacy
from pprint import pprint
from collections import Counter
import en_core_web_sm
from textblob import TextBlob 
nlp = en_core_web_sm.load()

# API Fetching for News,Jokes and Weather

In [47]:
def get_news(entity): 
    main_url = " https://newsapi.org/v1/articles?source=bbc-news&sortBy=top&apiKey=e3fd21c34e0a4062b9480ddaa91707cb"
    flag1=False
    flag2=False
    for name in entity["org"]:
        if "business" in name:
            flag1=True
            print("Top 10 business news :")
            main_url="https://newsapi.org/v2/top-headlines?country=in&category=business&apiKey=e3fd21c34e0a4062b9480ddaa91707cb"
            break
        if "sports" in name:
            flag1=True
            print("Top 10 sports news :")
            main_url="https://newsapi.org/v2/top-headlines?country=in&category=sports&apiKey=e3fd21c34e0a4062b9480ddaa91707cb"
            break
    if flag1==False:
        if(len(entity["person"])):
            flag2=True
            main_url="https://newsapi.org/v2/everything?q="+entity["person"][0]+"&sortBy=publishedAt&apiKey=e3fd21c34e0a4062b9480ddaa91707cb"
#https://newsapi.org/v2/everything?q=bitcoin&from=2019-04-09&sortBy=publishedAt&apiKey=e3fd21c34e0a4062b9480ddaa91707cb

#https://newsapi.org/v2/top-headlines?country=us&category=business&apiKey=API_KEY
#https://newsapi.org/v2/top-headlines?sources=techcrunch&apiKey=API_KEY
#https://newsapi.org/v2/everything?domains=wsj.com&apiKey=API_KEY

    open_bbc_page = requests.get(main_url).json() 
    article = open_bbc_page["articles"] 
    results = [] 
    for ar in article: 
                results.append(ar["title"]) 
    counter=1      
    for i in range(len(results)):
        if(flag1):
            print(i + 1, results[i])
            counter+=1
            if(counter==11):
                    break
        if(flag2):
            if(entity["person"][0].lower() in results[i].lower()):
                    print(counter, results[i])
                    counter+=1
                    if(counter==11):
                        break
def get_jokes(entity):
#     https://icanhazdadjoke.com/search?term=hipster
    main_url='https://icanhazdadjoke.com'
    flag1=False
    flag2=False
    if(len(entity["person"])):
            flag1=True
            main_url="https://icanhazdadjoke.com/search?term="+entity["person"][0]
    r = requests.get(main_url, headers={"Accept":"application/json"})
    raw_joke = r.json();
    print(raw_joke['joke'])

def get_weather(entity):
        app_id = "e884734dcec3248d3d1364baf22b5470"
        city_name=None
        country_code=None
        if(city_name is None):
                city_name = "delhi"
        if(country_code is None):        
                country_code = 'IND'
        complete_url = 'http://api.openweathermap.org/data/2.5/forecast?q=' + city_name + ',' + country_code + '&appid=' + app_id + '&mode=json&units=metric'
        response = requests.get(complete_url)  
        x = response.json() 
        if x["cod"] != "404": 
            humidity = []
            date = []
            temp = []
            desc = []
            pressure = []
            sea_level = []

            data =x
            print("\t" + "Details about the forcast of the city " + city_name + " of next 5 days " + "\n\n")
            for lists in data['list']:
                date.append(lists['dt_txt'])
                temp.append(lists['main']['temp'])
                desc.append(lists['weather'][0]['description'])

            print('{:^25}{:^20}{:^20}'.format("Date", "Description", "Temprature\n"))
            for i in range(len(date)):
                print('{:^25}{:^20}{:^20}'.format(str(date[i]), desc[i], str(temp[i])+" C\N{DEGREE SIGN}"))

        else: 
            print(" City Not Found ")     

# Entity recognition Module

In [48]:
 def get_entity(sentence):
    tool = language_check.LanguageTool('en-UK')
    text = sentence
    matches = tool.check(text)
    sentence=language_check.correct(text, matches)
    sentence=sentence.lower()
    nouns = [] 
    for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
             if ((pos == 'NN' or pos == 'NNP' or pos == 'NNS' or pos == 'NNPS' )and word!="joke"):
                    nouns.append(word)
    split_sentence=sentence.split()
    new_sentence=""
    for word in split_sentence:
        if word in nouns:
                new_sentence+=word.capitalize()
        else:
            new_sentence+=word
        new_sentence+=" "    
    doc=nlp(new_sentence)
    entity={}
    entity["person"]=[]
    entity["place"]=[]
    entity["date"]=[]
    entity["org"]=[]
    for X in doc.ents:
#         print(X.label_,X.text)
        if(X.label_=='PERSON'):
            entity["person"].append(X.text.lower())
        elif(X.label_=='GPE'):
            entity["place"].append(X.text.lower())
        elif(X.label_=='DATE'):
            entity["date"].append(X.text.lower())
        elif(X.label_=='ORG'):
            entity["org"].append(X.text.lower())
    return entity

In [49]:
def model(text,op):
    #model=build_model()
    #op=model.predict(text)
    if(op=="news"):
            entity=get_entity(text)
#             print(entity)
            get_news(entity)
    if(op=="weather"):
            entity=get_entity(text)
            get_weather(entity)
    if(op=="jokes"):
            entity=get_entity(text)
#             print(entity)
            get_jokes(entity)

In [51]:
text=input("")
model(text,"jokes")

nn
The word queue is ironic. It's just q with a bunch of silent letters waiting in line.
